In [ ]:
import pandas as pd
data = pd.read_excel('../56529_WoS_Paper.xlsx',sheet_name='TI')
corpus_for_word2vec=data.values.tolist()

In [ ]:
print(len(corpus_for_word2vec))

In [ ]:
def get_stopwords():
    stopwords = []
    with open('../Word_List/stopword_list.txt','r',encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace('\n','')
            stopwords.append(line)
    return stopwords
stopwords = get_stopwords()

def get_academic_words():
    academic_words = []
    with open('../Word_List/academic_word_list-570.txt','r',encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace('\n','')
            academic_words.append(line)
    return academic_words
academic_words = get_academic_words()

In [ ]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
def NLP_Prpcessing(text):
    text = str(text)
    # Remove non-English characters.
    text = re.sub(r'[^1-9a-zA-Z_]', " ", text)
    # Remove redundant spaces.
    text = ' '.join(text.split())
    text = text.lower()

    # Convert uppercase to lowercase and split words.
    tokenizer = RegexpTokenizer(r'\w+')         
    text = tokenizer.tokenize(text)
    
    # Remove stop words and common academic words.
    stopword = get_stopwords()  # Load stop word list.
    text = [word for word in text if word not in stopword]
    academic_word = get_academic_words()  # Load common academic word list.
    text = [word for word in text if word not in academic_word]

    # Remove words that are only one character.
    text = [token for token in text if not token.isnumeric()]
    text = [token for token in text if len(token) > 1]
    
    # Lemmatize the documents.
    lemmatizer = WordNetLemmatizer()
    '''
    text = [lemmatizer.lemmatize(token, pos = 'v') for token in text]
    text = [lemmatizer.lemmatize(token, pos = 'a') for token in text]
    '''
    text = [lemmatizer.lemmatize(token, pos = 'n') for token in text]
    
    return text

In [ ]:
for i in range(len(corpus_for_word2vec)):
    corpus_for_word2vec[i] = NLP_Prpcessing(corpus_for_word2vec[i])

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
bigram = Phrases(corpus_for_word2vec, min_count = 30, threshold=2)  
#trigram = Phrases(bigram[corpus_for_word2vec],min_count = 5, threshold=5)
#Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
#trigram_mod = Phraser(trigram)

for i in range(len(corpus_for_word2vec)):
    corpus_for_word2vec[i] = bigram_mod[corpus_for_word2vec[i]]

In [ ]:
print(len(corpus_for_word2vec))
print(corpus_for_word2vec[0])

In [ ]:
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence

word2vec_model = word2vec.Word2Vec(sentences = corpus_for_word2vec, size=300, min_count = 1,iter=300)
word_list = word2vec_model.wv.index2word

In [ ]:
word_list[:1000]

In [ ]:
print(word2vec_model.wv.similarity('deep_learning','machine_learning'))
print(word2vec_model.wv.similarity('face','machine_learning'))
print(word2vec_model.wv.similarity('face','facial'))
word2vec_model.wv.most_similar('facial')

In [ ]:
import smart_open
import gensim
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [ ]:
train_corpus = list(read_corpus('TI&AB.txt'))

In [ ]:
print(len(train_corpus))
#train_corpus[0]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(dm=0, vector_size = 300, epochs = 10)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples = model.corpus_count, epochs = model.epochs)

In [ ]:
'''
model.save('doc2vecModel.vec')

from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load('doc2vecModel.vec')
'''

In [ ]:
docvecs = model.docvecs
docvectors = []
#for i in range(len(docvecs)):
#[0,2753), [2753,6225), [6225,10137), [10137,14675), [14675,19878), [19878,25376), [25376,31657), [31657,38877), [38877,47540), [47540,56529)
for i in range(47540,56529):
    docvectors.append(docvecs[i])
len(docvectors)

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

classNumber = 24
# k = 12, 14, 14, 16, 17, 17, 19, 21, 23, 24
# k = 8, 9, 10, 11, 12, 12, 13, 14, 16, 16
kmean_model = KMeans(n_clusters = classNumber).fit(docvectors)
labels = kmean_model.labels_
cluster_centers = kmean_model.cluster_centers_
print(labels)
center_dict = Counter(labels)
#center_dict

In [ ]:
center_dict
topicSizeList = []
for key, value in sorted(center_dict.items(),key = lambda x:x[0],reverse = False): # Sort by Topic Tags
    topicSizeList.append(value)
print(topicSizeList)

In [ ]:
doccluster = [[]*classNumber for x in range(classNumber)]
for i in range(0, classNumber):
    for j in range(0, len(labels)):
        if i == labels[j]:
            doccluster[i].append(j+1+47540)
            # 0,2753,6225,10137,14675,19878,25376,31657,38877,47540

In [ ]:
# Generate topic vectors
import numpy as np
topicvector = [[]*classNumber for x in range(classNumber)]
for m in range(len(doccluster)):
    topicvector[m] = [0 for x in range(300)]
    for n in range(len(doccluster[m])):
        topicvector[m] = np.vstack((topicvector[m],docvecs[doccluster[m][n]-1]))
        topicvector[m] = np.mean(topicvector[m],axis=0)

In [ ]:
print(doccluster[0][:5])
len(doccluster)

In [ ]:
# Obtain the term composition of each topic
# topics: semantic network, c_topics: co-occurrence network
topics = [[]*classNumber for x in range(classNumber)]
c_topics = [[]*classNumber for x in range(classNumber)]
for i in range(len(doccluster)):
    for j in doccluster[i]:
        topics[i] += corpus_for_word2vec[j-1]
        c_topics[i].append(corpus_for_word2vec[j-1])        

In [ ]:

def get_commonwords():
    commonwords = []
    with open('CV_commonwords.txt','r',encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace('\n','')
            commonwords.append(line)
    return commonwords
commonwords = get_commonwords()

In [ ]:
def word_frequency_count(text):
    count_dict = {}
    # Count the number of each term
    text = [token for token in text if (token !='' and token not in commonwords)]
    for i in text:
        count_dict[i] = count_dict.get(i,0) + 1
    
    # Convert to a list
    count_dict = list(count_dict.items())
    count_dict.sort(key = lambda x:x[1], reverse = True)
   
    return count_dict

In [ ]:
#word_frequency_count(topics[1])  #[('deep_learning', 52), ('recognition', 39), ('video', 32)...]

In [ ]:
# Construct term co-occurrence matrix
def kwCoOccurrenceMatrix(terms, c_topics):
    edge = len(terms)       # The size of the matrix are the length of the term set
    matrix = np.zeros((edge, edge), dtype=str)
    appeardict = {}
    for i in terms:
        appearlist = []
        k = 0
        for j in c_topics:
            if i in j:
                appearlist.append(k)
            k += 1
        appeardict[i] = appearlist
    #return appeardict

    for row in range(len(terms)):
        # Iterate through the first row of the matrix, skipping the elements with subscript 0
        for col in range(len(terms)):
            # Iterate through the first column of the matrix, skipping the elements with subscript 0
            if col >= row:
                # Calculate only the upper half of the matrix
                if terms[row] == terms[col]:
                    matrix[col][row] = 0
                else:
                    counter = len(set(appeardict[terms[row]])&set(appeardict[terms[col]]))
                    matrix[col][row] = counter
            else:
                matrix[col][row]=matrix[row][col]
    return matrix            
    

In [ ]:
import networkx as nx
from openpyxl import Workbook
wb = Workbook()
topicSheet_kwDegreeCentrality = wb.create_sheet('topics_kwSemanticNet_Centrality',index=0)
topicSheet_kwCoOccurrence = wb.create_sheet('topics_kwCoOccurNet_Centrality',index=1)
topicSheet_kw_sheet = wb.create_sheet('topics_kw',index=2)
topic_vector_sheet = wb.create_sheet('topic vector', index=3)
#topicSheet_kwFre = wb.create_sheet('topics_kwFre',index=4)
#topicSheet_kwIntegratedCentrality = wb.create_sheet('topics_Integrated_Centrality', index=2)

topn = 30    # Consider only the top 30 keywords in terms of word frequency

for i in range(classNumber):
    # For each topic, construct term network
    termNetwork = nx.Graph()

    # Create Node
    term = word_frequency_count(topics[i]) 
    terms = []
    termsWithFre = []
    for m in range(topn):
        if term[m][0] not in commonwords:
            terms.append(term[m][0])   # Store nodes
            termsWithFre.append(str(term[m][1])+'*'+term[m][0])   # Store high frequency terms 
    termNetwork.add_nodes_from(terms)

    # Create edges
    edgeList = []
    similarityMatrix = np.zeros(shape=(topn,topn))
    for m in range(topn):
         for n in range(topn):
            similarityMatrix[m][n] = 0.5 + 0.5*word2vec_model.wv.similarity(terms[m],terms[n])  # Normalize to [0,1]
    upper_quartile = np.quantile(similarityMatrix, 0.75, interpolation = 'higher')      # Prune according to the upper quartile
    similarityMatrix[similarityMatrix < upper_quartile] = 0

    for m in range(topn):
         for n in range(topn):
            if 1 > similarityMatrix[m][n] > 0:
                edgeList.append((terms[m],terms[n],similarityMatrix[m][n]))
    termNetwork.add_weighted_edges_from(edgeList)


    for k in range(len(topicvector[i])):
        topic_vector_sheet.cell(row=i+1, column=k+1, value = topicvector[i][k])

    topicSheet_kwDegreeCentrality.cell(row=i+1,column=1,value='topic '+ str(i+1))
    
    termsList = []
    for key, value in sorted(nx.degree_centrality(termNetwork).items(),key=lambda x:x[1],reverse=True):   
    # the effect of betweenness_centrality or eigenvector_centrality is not very good.
        termsList.append((str(round(value,2))+'*'+key))
    topicSheet_kwDegreeCentrality.cell(row=i+1,column=2,value=', '.join(termsList))    


    # For each topic, construct semantic network
    c_termsNetwork = nx.Graph()
    c_term = word_frequency_count(topics[i])    
    c_terms = []
     
    # Create Node
    for m in range(topn):
        if c_term[m][0] not in commonwords:
            c_terms.append(c_term[m][0])   # Store nodes
    c_termsNetwork.add_nodes_from(c_terms)
    
    # Create edges
    c_edgeList = []
    kwMatrix = kwCoOccurrenceMatrix(c_terms, c_topics[i])
    for m in range(topn):
        for n in range(topn):
            if int(kwMatrix[m][n])>0:
                c_edgeList.append((c_terms[m],c_terms[n],kwMatrix[m][n]))
    c_termsNetwork.add_weighted_edges_from(c_edgeList)


    topicSheet_kwCoOccurrence.cell(row=i+1,column=1,value='topic '+ str(i+1))
    c_termsList = []
    for key, value in sorted(nx.degree_centrality(c_termsNetwork).items(),key=lambda x:x[1],reverse=True):   
        c_termsList.append((str(round(value,2))+'*'+key))
        #topic = ', '.join(c_termsList)
    topicSheet_kwCoOccurrence.cell(row=i+1,column=2,value=', '.join(c_termsList))    

    '''
    # Sum of two degree centrality
    termNetworkDict = nx.degree_centrality(termNetwork)
    c_termNetworkDict = nx.degree_centrality(c_termsNetwork)
    integratedCentralityList = []

    for key,value in termNetworkDict.items():
        c_termNetworkDict[key] += value
    for key, value in sorted(c_termNetworkDict.items(),key=lambda x:x[1],reverse=True):   
        integratedCentralityList.append((str(round(value,2))+'*'+key))
    '''

    topicSheet_kw_sheet.cell(row=i+1,column=1,value='topic '+ str(i+1))
    topicSheet_kw_sheet.cell(row=i+1,column=2,value=', '.join(termsList[:3])) 
    topicSheet_kw_sheet.cell(row=i+1,column=3,value=', '.join(c_termsList[:3]))  

wb.save('TOPICS_termDegree&Fre.xlsx')

In [ ]:
termNetwork.get_edge_data('cnn','object_detection')